In [134]:
import aoc.common.process_input
import aoc.year2022.day15

In [135]:
fileContent = aoc.common.process_input.read_file("input\\year2022\\day15\\example.txt")        
input_list = aoc.common.process_input.to_function_list(fileContent, str)

In [136]:
# Sensor at x=2, y=18: closest beacon is at x=-2, y=15
def process_line(line):
    parts = line.split(' ')

    sensor_x_str = parts[2]
    sensor_x = int(sensor_x_str[2:-1])
    sensor_y_str = parts[3]
    sensor_y = int(sensor_y_str[2:-1])

    beacon_x_str = parts[8]
    beacon_x = int(beacon_x_str[2:-1])
    beacon_y_str = parts[9]
    beacon_y = int(beacon_y_str[2:])
    
    return (sensor_x, sensor_y), (beacon_x, beacon_y)

process_line("Sensor at x=2, y=18: closest beacon is at x=-2, y=15")

((2, 18), (-2, 15))

In [137]:
sensor_beacon_pairs = []
for input in input_list:
    sensor, beacon = process_line(input)
    sensor_beacon_pairs.append((sensor, beacon))

y_line = 0 # for the example, that's another thing we need to make sure it is passed

# need to check if the circle with sensor as center, and radius the vector "sensor-beacon" intersects line Y=YLINE

cross_pairs_and_md = []
for sbp in sensor_beacon_pairs:
    s = sbp[0]
    b = sbp[1]
    md = abs(s[0]-b[0]) + abs(s[1]-b[1])
    if s[1] < y_line:
        if s[1] + md > y_line:
            # print(sbp, "crosses line", y_line, "distance", md)
            cross_pairs_and_md.append((sbp[0], sbp[1], md))
    elif s[1] > y_line:
        if s[1] - md < y_line:
            # print(sbp, "crosses line", y_line, "distance", md)
            cross_pairs_and_md.append((sbp[0], sbp[1], md))
    else: # s[1] == y_line
        # print(sbp, "crosses line", y_line, "distance", md)
        cross_pairs_and_md.append((sbp[0], sbp[1], md))

print(cross_pairs_and_md)

intersect_x = []
intersect_x_pairs = []
for cp in cross_pairs_and_md:
    s = cp[0]
    b = cp[1]
    md = cp[2]

    px_1 = - (md - abs(y_line - s[1])) + s[0]
    px_2 = (md - abs(y_line - s[1])) + s[0]
    intersect_x.append(px_1)
    intersect_x.append(px_2)
    intersect_x_pairs.append((px_1, px_2))
    
intersect_x.sort()
print(intersect_x_pairs)
min_x = intersect_x[0]
max_x = intersect_x[-1]

[((13, 2), (15, 3), 3), ((8, 7), (2, 10), 9), ((2, 0), (2, 10), 10), ((20, 1), (15, 3), 7)]
[(12, 14), (6, 10), (-8, 12), (14, 26)]


In [138]:
beacons_in_y_line = []
for sbp in sensor_beacon_pairs:
    b = sbp[1]
    if b[1] == y_line:
        if b[0] not in beacons_in_y_line:
            beacons_in_y_line.append(b[0])

print(beacons_in_y_line)

[]


In [139]:
result = abs(min_x) + abs(max_x) + 1 - len(beacons_in_y_line)
print(result)

35


In [140]:
fileContent = aoc.common.process_input.read_file("input\\year2022\\day15\\input.txt")        
input_list = aoc.common.process_input.to_function_list(fileContent, str)

sensor_beacon_pairs = []
for input in input_list:
    sensor, beacon = process_line(input)
    sensor_beacon_pairs.append((sensor, beacon))
    
pairs_and_md = []
for sbp in sensor_beacon_pairs:
    s = sbp[0]
    b = sbp[1]
    md = abs(s[0]-b[0]) + abs(s[1]-b[1])
    pairs_and_md.append((s, b, md))   

for pm in pairs_and_md:
    print(pm[0], pm[1], pm[2])

(3907621, 2895218) (3790542, 2949630) 171491
(1701067, 3075142) (2275951, 3717327) 1217069
(3532369, 884718) (2733699, 2000000) 1913952
(2362427, 41763) (2999439, -958188) 1636963
(398408, 3688691) (2275951, 3717327) 1906179
(1727615, 1744968) (2733699, 2000000) 1261116
(2778183, 3611924) (2275951, 3717327) 607635
(2452818, 2533012) (2733699, 2000000) 813893
(88162, 2057063) (-109096, 390805) 1863516
(2985370, 2315046) (2733699, 2000000) 566717
(2758780, 3000106) (3279264, 2775610) 744980
(3501114, 3193710) (3790542, 2949630) 533508
(313171, 1016326) (-109096, 390805) 1047788
(3997998, 3576392) (3691556, 3980872) 710922
(84142, 102550) (-109096, 390805) 481493
(3768533, 3985372) (3691556, 3980872) 81477
(2999744, 3998031) (3691556, 3980872) 708971
(3380504, 2720962) (3279264, 2775610) 155888
(3357940, 3730208) (3691556, 3980872) 584280
(1242851, 838744) (-109096, 390805) 1799886
(3991401, 2367688) (3790542, 2949630) 782801
(3292286, 2624894) (3279264, 2775610) 163738
(2194423, 3990859)